# Forex Pip Value Calculator
## Overview:

This Forex Pip Calculator helps traders calculate the value of a pip for any currency pair based on the current exchange rate and lot size. It provides quick and accurate results, assisting traders in determining potential profits or losses in Forex trading.

## Features:
    
         * Currency Pair Support: Calculate pip values for any currency pair, including pairs involving JPY.
         * Lot Size Customization: Input different lot sizes (e.g., 0.1 lot, 1.0 lot) for accurate calculations.
         * Account Currency Support: The calculator adjusts pip value based on the trader’s account currency (USD, EUR, GBP, etc.).
         * Exchange Rate Integration: Takes current exchange rates as inputs to provide up-to-date calculations.

## Formula for Pip Value Calculation


For most currency pairs:
$$
\text{Pip Value} = \frac{1 \, \text{Pip}}{\text{Exchange Rate}} \times \text{Lot Size}
$$

For cross-currency pairs (if the account currency is not the quote currency):
$$
\text{Pip Value} = \frac{1 \, \text{Pip}}{\text{Exchange Rate of Pair}} \times \text{Lot Size} \times \text{Exchange Rate of Quote Currency to Account Currency}
$$

## Example

### Inputs:

Currency Pair: EUR/USD
Lot Size: 1.0 lot
Account Currency: EUR
Exchange Rate: 1.18

### Output:

Pip Value for EUR/USD
:
0.00009
 
EUR
Pip Value for EUR/USD:0.00009EUR


In [4]:
import yfinance as yf

def get_live_exchange_rate(currency_pair):
    """
    Fetch live exchange rate using Yahoo Finance for the given currency pair.
    Example: 'EURUSD=X' for EUR/USD.
    
    Parameters:
    currency_pair (str): The currency pair symbol (e.g., 'EURUSD=X', 'GBPUSD=X').
    
    Returns:
    float: The live exchange rate for the currency pair.
    """
    pair = f"{currency_pair}=X"
    ticker = yf.Ticker(pair)
    data = ticker.history(period='1d')
    live_rate = data['Close'].iloc[-1]
    return live_rate

def pip_value(currency_pair, lot_size, exchange_rate):
    """
    Calculate the value of 1 pip for a given currency pair based on the live exchange rate.
    
    Parameters:
    currency_pair (str): The currency pair (e.g., 'EURUSD').
    lot_size (float): The trade size in lots (e.g., 1.0 lot).
    exchange_rate (float): The current exchange rate of the currency pair.
    
    Returns:
    float: The pip value.
    """
    
    # Automatically detect if the pair involves JPY and adjust pip value accordingly
    pip = 0.01 if 'JPY' in currency_pair else 0.0001
    
    # Basic pip value calculation for 1 lot (standard 100,000 units)
    pip_value = pip / exchange_rate * lot_size
    
    return pip_value

def calculate_profit(investment_amount, pip_value, pip_movement):
    """
    Calculate the profit or loss based on investment, pip value, and pip movement.
    
    Parameters:
    investment_amount (float): The total investment amount.
    pip_value (float): The value of 1 pip.
    pip_movement (int): The number of pips the market moved.
    
    Returns:
    float: The profit or loss.
    """
    # Profit or loss = pip_value * investment * pip_movement
    profit_loss = pip_value * investment_amount * pip_movement
    return profit_loss

# Interactive inputs
currency_pair = input("Enter the currency pair (e.g., EURUSD, GBPJPY): ")
lot_size = float(input("Enter the lot size (e.g., 1.0, 0.1, 0.01): "))
investment_amount = float(input("Enter the investment amount (in the account currency): "))
pip_movement = int(input("Enter the expected pip movement (positive for gain, negative for loss): "))

# Step 1: Fetch the live exchange rate
exchange_rate = get_live_exchange_rate(currency_pair)

# Step 2: Calculate pip value based on live exchange rate and lot size
pip_val = pip_value(currency_pair, lot_size, exchange_rate)

# Step 3: Calculate profit based on pip value, investment amount, and pip movement
profit = calculate_profit(investment_amount, pip_val, pip_movement)

# Output the results
print(f"Live Exchange Rate for {currency_pair}: {exchange_rate}")
print(f"Pip Value for {currency_pair} with lot size {lot_size}: {pip_val:.5f}")
print(f"Profit for {pip_movement} pip movement with investment of {investment_amount}: {profit:.2f} {currency_pair[:3]}")


Live Exchange Rate for EURJPY: 162.89700317382812
Pip Value for EURJPY with lot size 1.0: 0.00006
Profit for 20 pip movement with investment of 20000.0: 24.56 EUR


# Triangular Arbitrage 

In [3]:
import yfinance as yf
import pandas as pd
import os 
# Define currency pairs for triangular arbitrage
CURRENCY_PAIRS = {
    'EURUSD=X': 'EUR/USD',
    'USDJPY=X': 'USD/JPY',
    'EURJPY=X': 'EUR/JPY'
}

# Function to fetch historical data using yfinance
def fetch_historical_data(pair):
    df = yf.download(pair, period="1y", interval="1d")
    
    if df.empty:
        print(f"Error fetching data for {pair}")
        return None
    
    df = df.reset_index()
    df['Pair'] = pair
    return df[['Date', 'Pair', 'Close']]

# Function to detect arbitrage opportunities
def detect_arbitrage_opportunities(eur_usd, usd_jpy, eur_jpy):
    opportunities = []
    
    # Merge data on Date for comparison
    merged_df = eur_usd.merge(usd_jpy, on='Date', suffixes=('_EURUSD', '_USDJPY')).merge(eur_jpy, on='Date', suffixes=('', '_EURJPY'))
    
    for _, row in merged_df.iterrows():
        # Calculate the expected EUR/JPY rate based on EUR/USD and USD/JPY
        expected_eur_jpy = row['Close_EURUSD'] * row['Close_USDJPY']
        actual_eur_jpy = row['Close']
        
        # Check if there's an arbitrage opportunity
        if abs(actual_eur_jpy - expected_eur_jpy) > 0.01:  # Arbitrage threshold
            opportunities.append({
                'Date': row['Date'],
                'Expected_EURJPY': expected_eur_jpy,
                'Actual_EURJPY': actual_eur_jpy,
                'Opportunity': 'YES'
            })
    
    return pd.DataFrame(opportunities)

# Main function to list past arbitrage opportunities
def list_past_arbitrage_opportunities():
    print("Fetching historical data...")
    
    # Fetch historical data for the three currency pairs
    eur_usd = fetch_historical_data('EURUSD=X')
    usd_jpy = fetch_historical_data('USDJPY=X')
    eur_jpy = fetch_historical_data('EURJPY=X')
    
    if eur_usd is None or usd_jpy is None or eur_jpy is None:
        print("Failed to fetch data for one or more pairs. Exiting.")
        return
    
    print("Detecting arbitrage opportunities...")
    
    # Detect arbitrage opportunities
    opportunities = detect_arbitrage_opportunities(eur_usd, usd_jpy, eur_jpy)
    
    if opportunities.empty:
        print("No arbitrage opportunities found.")
    else:
        print("Arbitrage opportunities detected:")
        print(opportunities)
        
        # Save opportunities to a CSV file
        opportunities.to_csv('/Users/jebinlarosh/Downloads/arbitrage_opportunities.csv', index=False)
        print("Opportunities saved to 'arbitrage_opportunities.csv'.")

if __name__ == '__main__':
    list_past_arbitrage_opportunities()


Fetching historical data...
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Detecting arbitrage opportunities...
Arbitrage opportunities detected:
          Date  Expected_EURJPY  Actual_EURJPY Opportunity
0   2023-10-11       157.616904     157.598999         YES
1   2023-10-12       158.340050     158.328995         YES
2   2023-10-13       157.829837     157.811996         YES
3   2023-10-16       157.366665     157.354996         YES
4   2023-10-18       158.341082     158.309998         YES
..         ...              ...            ...         ...
154 2024-09-27       162.154058     162.126007         YES
155 2024-10-01       160.044117     160.031998         YES
156 2024-10-03       161.951166     161.917999         YES
157 2024-10-08       162.601246     162.574005         YES
158 2024-10-09       162.960719     162.92199